<a href="https://colab.research.google.com/github/ForestPearson/CS410-510-NLP-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
print('bot text?')